In [1]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import pandas as pd
import seaborn as sns
import statsmodels.api as sm

# 1.1

In [2]:
# Save data in array
with open("framingham.csv") as file_name:
    df = pd.read_csv(file_name, encoding="latin1")

In [3]:
# It's a good practice to make a copy of the 
# original dataframe before modifying it
df_copy = df.copy()
df_copy = df_copy.astype(float)
df_copy.fillna(df_copy.mean(), inplace=True)
column_headers = list(df_copy.columns.values)
# Save data in an array
df_array = df_copy.values

In [4]:
# Convert the array to a Pandas DataFrame
df = pd.DataFrame(df_array, columns=column_headers)

In [5]:
column_headers

['male',
 'age',
 'education',
 'currentSmoker',
 'cigsPerDay',
 'BPMeds',
 'prevalentStroke',
 'prevalentHyp',
 'diabetes',
 'totChol',
 'sysBP',
 'diaBP',
 'BMI',
 'heartRate',
 'glucose',
 'TenYearCHD']

# 1.2

In [6]:
'''
Total variables:
'male','age','education','currentSmoker','cigsPerDay','BPMeds','prevalentStroke',
'prevalentHyp','diabetes','totChol','sysBP','diaBP','BMI','heartRate','glucose',
'TenYearCHD'
Variables needed: Unknown, perform forward stepwise
Dependent variable: TenYearCHD

'''
# Drop the "male" column
df.drop("male", axis=1, inplace=True)
# Convert all columns to numeric
df = df.apply(pd.to_numeric)


# Too many variables, but we need to make sure which ones to use.
# Apply stepwise regression to figure out p-value
# Define dependent variable and independent variables
y = df["TenYearCHD"]
X = df.drop(["TenYearCHD"], axis=1)

In [7]:
# Perform stepwise regression
def forward_stepwise(X, y, initial_list=[], threshold_in=0.01, verbose=True):
    included = list(initial_list)
    excluded = [col for col in X.columns if col not in included]
    while True:
        changed = False
        # Forward step
        excluded_dict = {}
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(X[included + [new_column]])).fit()
            p_value = model.pvalues[new_column]
            excluded_dict[new_column] = p_value
        best_column, best_pvalue = min(excluded_dict.items(), key=lambda x: x[1])
        if best_pvalue < threshold_in:
            included.append(best_column)
            excluded.remove(best_column)
            if verbose:
                print("Add {:30} with p-value {:.6}".format(best_column, best_pvalue))
            changed = True
        # Termination condition: no more variables can be added
        if not changed:
            break
    return included

In [8]:
# Run the forward stepwise function 
# These will be the variables used for the model
selected_columns = forward_stepwise(X, y)

Add age                            with p-value 6.84501e-50
Add sysBP                          with p-value 1.14106e-20
Add cigsPerDay                     with p-value 1.20477e-12
Add glucose                        with p-value 6.91293e-09
Add prevalentStroke                with p-value 0.00196294


In [9]:
# Select only the five variables chosen by stepwise regression
X = df[['age', 'sysBP', 'cigsPerDay', 'glucose', 'prevalentStroke']].to_numpy()

# Extract the target variable
y = df['TenYearCHD'].astype(int)

In [10]:
s = pd.Series(y)
s_array = s.to_numpy()
s_reshaped = s_array.reshape(-1, 1)
y = s_reshaped

In [11]:
# Training variables
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 1.3

In [12]:
def sigmoid(z):
    """
    Compute the sigmoid of a scalar or an array.
    """
    return 1 / (1 + np.exp(-z))

In [13]:
def logistic_regression(X, y, learning_rate=0.01, num_iterations=1000):
    """
    Implement logistic regression without using any external libraries.

    Arguments:
    X -- input data, shape (m, n)
    y -- true "label" vector (containing 0s and 1s), shape (m, 1)
    learning_rate -- learning rate for gradient descent
    num_iterations -- number of iterations of gradient descent

    Returns:
    w -- weights, shape (n, 1)
    b -- bias, a scalar
    """
    
    # Initialize parameters
    m, n = X.shape
    w = np.zeros((n, 1))
    b = 0

    # Gradient descent loop
    for i in range(num_iterations):
        
        # Forward propagation
        z = np.dot(X, w) + b
        A = sigmoid(z)
        cost = -1/m * np.sum(y*np.log(A) + (1-y)*np.log(1-A))

        # Backward propagation
        dw = 1/m * np.dot(X.T, (A - y))
        db = 1/m * np.sum(A - y)

        # Update parameters
        w = w - learning_rate * dw
        b = b - learning_rate * db
        
        # Print cost every 100 iterations
        if i % 100 == 0:
            print("Cost after iteration %i: %f" % (i, cost))

    return w, b

def logistic_regression_validation(X, y, learning_rate=0.01, num_iterations=1000):
    """
    Implement logistic regression without using any external libraries.

    Arguments:
    X -- input data, shape (m, n)
    y -- true "label" vector (containing 0s and 1s), shape (m, 1)
    learning_rate -- learning rate for gradient descent
    num_iterations -- number of iterations of gradient descent

    Returns:
    w -- weights, shape (n, 1)
    b -- bias, a scalar
    """
    
    # Initialize parameters
    m, n = X.shape
    w = np.zeros((n, 1))
    b = 0

    # Gradient descent loop
    for i in range(num_iterations):
        
        # Forward propagation
        z = np.dot(X, w) + b
        A = sigmoid(z)
        cost = -1/m * np.sum(y*np.log(A) + (1-y)*np.log(1-A))

        # Backward propagation
        dw = 1/m * np.dot(X.T, (A - y))
        db = 1/m * np.sum(A - y)

        # Update parameters
        w = w - learning_rate * dw
        b = b - learning_rate * db

    return w, b

In [14]:
weights, bias = logistic_regression(X_train, y_train, learning_rate=0.0001, num_iterations=1000)

Cost after iteration 0: 0.693147
Cost after iteration 100: 0.453698
Cost after iteration 200: 0.453632
Cost after iteration 300: 0.453595
Cost after iteration 400: 0.453573
Cost after iteration 500: 0.453558
Cost after iteration 600: 0.453548
Cost after iteration 700: 0.453541
Cost after iteration 800: 0.453536
Cost after iteration 900: 0.453531


In [15]:
def predict(X, weights, bias):
    z = np.dot(X, weights) + bias
    y_pred = sigmoid(z)
    y_pred_class = (y_pred > 0.5).astype(int)
    return y_pred_class

# Make predictions on the test set
y_pred = predict(X_test, weights, bias)

# Compute the accuracy
accuracy = (1 - np.mean(np.abs(y_pred - y_test))) * 100

print("Accuracy:", accuracy, "%")

Accuracy: 85.37735849056604 %


# 1.4

In [16]:
# Define the number of folds
k = 5

# Split the data into k folds
kf = KFold(n_splits=k)

# Initialize a list to store the accuracy scores
scores = []

# Loop over each fold
for train_index, test_index in kf.split(X):
    # Split the data into training and testing sets
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Train the logistic regression model
    weights, bias = logistic_regression(X_train, y_train, learning_rate=0.0001, num_iterations=100)
    
    # Make predictions on the test set
    y_pred = predict(X_test, weights, bias)
    
    # Compute the accuracy
    accuracy = (1 - np.mean(np.abs(y_pred - y_test))) * 100
    
    # Append the accuracy score to the list
    scores.append(accuracy)

# Compute the average accuracy over all folds
avg_accuracy = np.mean(scores)

print("Average accuracy:", avg_accuracy, "%")

Cost after iteration 0: 0.693147
Cost after iteration 0: 0.693147
Cost after iteration 0: 0.693147
Cost after iteration 0: 0.693147
Cost after iteration 0: 0.693147
Average accuracy: 84.80419237709118 %


In [17]:
def cross_validate_poly(X, y, k=5, learning_rate=0.01, num_iters=1000, degree_range=range(1, 6)):
    m, n = X.shape
    fold_size = int(m/k)
    best_degree = None
    best_accuracy = 0
    
    for degree in degree_range:
        X_poly = np.ones((m, 1))
        for d in range(1, degree+1):
            X_poly = np.hstack((X_poly, np.power(X, d)))
            
        for i in range(k):
            X_val = X_poly[i*fold_size:(i+1)*fold_size, :]
            y_val = y[i*fold_size:(i+1)*fold_size]
            X_train = np.vstack((X_poly[:i*fold_size, :], X_poly[(i+1)*fold_size:, :]))
            y_train = np.concatenate((y[:i*fold_size], y[(i+1)*fold_size:]))
            
            weights, bias = logistic_regression_validation(X_train, y_train, learning_rate, num_iters)
            
            y_pred = predict(X_val, weights, bias)
            
            accuracy = np.mean(y_pred == y_val)
            
            if accuracy > best_accuracy:
                best_degree = degree
                best_accuracy = accuracy
                
    return best_degree, best_accuracy  

In [18]:
best_degree, best_accuracy = cross_validate_poly(X, y)
print('Best Degree: ', best_degree)
print('Best Accuracy: ', best_accuracy)

C:\Users\Gamer\AppData\Local\Temp\ipykernel_2652\2804211525.py:69: RuntimeWarning: divide by zero encountered in log
  cost = -1/m * np.sum(y*np.log(A) + (1-y)*np.log(1-A))
C:\Users\Gamer\AppData\Local\Temp\ipykernel_2652\2804211525.py:69: RuntimeWarning: invalid value encountered in multiply
  cost = -1/m * np.sum(y*np.log(A) + (1-y)*np.log(1-A))
C:\Users\Gamer\AppData\Local\Temp\ipykernel_2652\2469156700.py:5: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


Best Degree:  5
Best Accuracy:  0.8654073199527745


# 1.5

In [19]:
print("La regresion logistica aplicada al modelo fue efectiva, tieniedo un \
acierto del 85%. La validacion cruzada, con un grado de polinomio 5, \
tuvo un acierto del 86%. Las 5 variables usadas para este modelo \
fueron elegidas por un forward stepwise, una tecnica usada en modelos \
estadisticos para encontrar el mejor subset de variables predictoras para \
un modelo de regresion lineal. La meta es poder encontrar las variables \
que aporten mejor rendimiento de prediccion, y entre menos variables, mejor. \
Es importante mencionar que el 85% a 86% de acierto, en el campo de medicina, \
es bastante bajo. La vida de personas depende de estos modelos de \
predicciones. Pero en general, el concepto de la regresion logistica \
fue aprendido y puesto en practica de manera efectiva. Sin embargo, hay \
librerias de sklearn que hacen de esta regresion logistica mas efectiva \
y exacta.")

La regresion logistica aplicada al modelo fue efectiva, tieniedo un acierto del 85%. La validacion cruzada, con un grado de polinomio 5, tuvo un acierto del 86%. Las 5 variables usadas para este modelo fueron elegidas por un forward stepwise, una tecnica usada en modelos estadisticos para encontrar el mejor subset de variables predictoras para un modelo de regresion lineal. La meta es poder encontrar las variables que aporten mejor rendimiento de prediccion, y entre menos variables, mejor. Es importante mencionar que el 85% a 86% de acierto, en el campo de medicina, es bastante bajo. La vida de personas depende de estos modelos de predicciones. Pero en general, el concepto de la regresion logistica fue aprendido y puesto en practica de manera efectiva. Sin embargo, hay librerias de sklearn que hacen de esta regresion logistica mas efectiva y exacta.
